In [1]:
import os
import pathlib
import numpy as np
import cv2
import glob
import shutil
from matplotlib import pyplot as plt
import random

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [3]:
from google.colab import drive
drive.mount("/content/drive")
!ls

data = pathlib.Path('/content/drive/My Drive/Intermediate_Assignment_Dataset') 
count = len(list(data.glob('*.pgm')))
base_dir = str(data)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive  sample_data


In [4]:
classes = ['YesSunglasses', 'NoSunglasses']
new_dir = "/content/sample_data/Intermediate_Assignment_Dataset"

In [8]:
#Creating a copy of images in a temporary location in collab
#It'll take a minute or two

for cl in classes:
    code = base_dir + '/' + cl
    images = glob.glob(code + '*.pgm')
    print('There are total {} images of {}'.format(len(images),cl))

    for i in images:
        if not os.path.exists(os.path.join(new_dir, cl)):
            os.makedirs(os.path.join(new_dir, cl))
        shutil.copy(i, os.path.join(new_dir, cl))

There are total 311 images of YesSunglasses
There are total 313 images of NoSunglasses


In [9]:
all_imgs = []
imgshp = 50
for cl in classes:
  path = os.path.join(new_dir,cl)
  class_num = classes.index(cl)
  for im in os.listdir(path):
    try:
      img = cv2.imread(os.path.join(path,im),0)
      img = cv2.resize(img, (imgshp,imgshp))

      all_imgs.append([img,class_num])
    except Exception as e:
      pass
print(len(all_imgs))

624


In [10]:
#Shuffling the data

random.shuffle(all_imgs)

In [11]:
features = []
labels = []

for feature,label in all_imgs:
  features.append(feature)
  labels.append(label)


features = np.array(features).reshape(-1, imgshp, imgshp, 1) 
labels = np.array(labels)

features = features/255.0

In [12]:
#Design and compilation of the model

model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape = features.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [13]:
model.fit(features, labels, batch_size=100, epochs = 15, validation_split = 0.2)

Epoch 1/15
5/5 [==============================] - 2s 442ms/step - loss: 0.6979 - accuracy: 0.4930 - val_loss: 0.6832 - val_accuracy: 0.4640
Epoch 2/15
5/5 [==============================] - 2s 405ms/step - loss: 0.6518 - accuracy: 0.7615 - val_loss: 0.6921 - val_accuracy: 0.4480
Epoch 3/15
5/5 [==============================] - 2s 404ms/step - loss: 0.5824 - accuracy: 0.7094 - val_loss: 0.5458 - val_accuracy: 0.8560
Epoch 4/15
5/5 [==============================] - 2s 404ms/step - loss: 0.4730 - accuracy: 0.8176 - val_loss: 0.4847 - val_accuracy: 0.7920
Epoch 5/15
5/5 [==============================] - 2s 407ms/step - loss: 0.4059 - accuracy: 0.8277 - val_loss: 0.4006 - val_accuracy: 0.8400
Epoch 6/15
5/5 [==============================] - 2s 410ms/step - loss: 0.3351 - accuracy: 0.8537 - val_loss: 0.4533 - val_accuracy: 0.7360
Epoch 7/15
5/5 [==============================] - 2s 442ms/step - loss: 0.2855 - accuracy: 0.8938 - val_loss: 0.3040 - val_accuracy: 0.8640
Epoch 8/15
5/5 [====